## <font color='#4285f4'>Overview</font>

This notebook uses Gemini to analyze marketing campaigns and identify ideal customer segments for Chocolate AI. You can start with either a campaign description or a video ad, and Gemini will recommend customer segments to target based one 1/ a predefined set of segments, and 2/ a dynamic segment that doesn’t already exist. The notebook shows how you can easily query complex segment data using JSON, and it demonstrates how to find matches for a net new segment using vector search in BigQuery.

Process Flow: 
1. Retrieve pre-defined customer segments from a BigQuery table (${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments)
2. Text-based Analysis:
    - Fetch the details of the latest generated marketing campaign from another BigQuery table (${project_id}.${bigquery_chocolate_ai_dataset}.campaign)
    - Construct a detailed prompt for Gemini LLM, providing campaign information and asking it to identify relevant pre-defined and novel customer segments along with explanations.
    - Process the LLM response and extract the suggested segments and reasoning.
    - Query BigQuery to count customers matching the recommended pre-defined segments.
3. Video-based Analysis:
    - Upload a sample advertisement video to a Google Cloud Storage bucket.
    - Call multimodal Gemini, providing the video URI and a prompt to identify customer segments likely receptive to the ad.
    - Extract and displays the recommended pre-defined and novel segments, along with explanations.
4. Increase reach with dynamic segments powered by vector search
    - Extract keywords from the LLM-generated ideal segment description.
    - Perform vector search on customer profiles using these keywords to retrieve a list of matching customers (i.e. top 100).

Author: Paul Ramsey

## <font color='#4285f4'>License</font>

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

## <font color='#4285f4'>Initialize</font>

### Pip Installs and Imports

In [ ]:
from IPython.display import HTML
from functools import reduce
import google.auth
import requests
import json
import markdown

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
# Update these variables to match your environment
location="us-central1" # Your region
bigquery_location = "${bigquery_location}" # Must be "us" or "eu"

### Do not change the values in this cell below this line ###
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

bucket_name = "${chocolate_ai_bucket}"

print(f"project_id = {project_id}")
print(f"user = {user}")
print(f"location = {location}")
print(f"bigquery_location = {bigquery_location}")
print(f"bucket_name = {bucket_name}")

### Helper Methods

#### RetryCondition(error)

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-2.0-flash", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-2.0-flash"

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-2.0-flash":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM_Multimodal(
    text_prompt, video_uri, model="gemini-2.0-flash", response_schema=None, temperature=1, topP=1, topK=32
):
    """
    Calls the Gemini API with a text prompt and an image prompt.

    Args:
      text_prompt: The text prompt.
      image_data: A BytesIO object containing the image data.
      model: The Gemini model to use.
      response_schema: Optional response schema.
      temperature: Temperature parameter for the model.
      topP: Top-p parameter for the model.
      topK: Top-k parameter for the model.

    Returns:
      The Gemini response as a string.
    """

    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    access_token = creds.token

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + access_token
    }

    url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"


    generation_config = {
        "temperature": temperature,
        "topP": topP,
        "maxOutputTokens": 8192,
        "candidateCount": 1,
        "responseMimeType": "application/json"  #Invalid for multi-modal responses
    }

    if response_schema is not None:
        generation_config["responseSchema"] = response_schema

    if model in ["gemini-2.0-flash"]:
        generation_config["topK"] = topK

    # Construct the payload with the image URI
    payload = {
        "contents": [{
            "role": "user",
            "parts": [
                {
                    "text": text_prompt
                },
                {
                    "fileData": {
                        "fileUri": video_uri,
                        "mimeType": "video/mp4"
                    }
                }
            ]
        }],
        "generation_config": generation_config,
        "safety_settings": {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_ONLY_HIGH"
        }
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        return json.loads(response.text)
    else:
        raise RuntimeError(
            f"Error with prompt:'{text_prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
        )


#### Run Query()

In [ ]:
def RunQuery(sql, job_config = None):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    if job_config == None:
      job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

#### PrettyPrintJson(json_string)

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  #print(json_formatted_str)
  return json.dumps(json_object)

#### DisplayMarkdown()

In [ ]:
def DisplayMarkdown(text):
  """
  Displays text in markdown/HTML format in a Colab notebook.

  Args:
    text: The text to display. Can be plain text or Markdown.
  """

  formatted_text = markdown.markdown(text)  # Convert to HTML if necessary
  display(HTML(formatted_text))


### Campaign Methods

#### GetPreDefinedSegments()

In [ ]:
def GetPreDefinedSegments():
  sql = f"""SELECT JSON_OBJECT(
    'pre_defined_customer_segments', JSON_ARRAY (
      (
        --benefits_sought
        SELECT JSON_OBJECT('benefits_sought', ARRAY_AGG(DISTINCT segment))
        FROM (
          SELECT DISTINCT  benefits_sought AS segments
          FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
        ), UNNEST(SPLIT(segments, ',')) AS segment
      ),
      (
        --browsing_behavior
        SELECT JSON_OBJECT('browsing_behavior', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
        FROM (
          SELECT DISTINCT  browsing_behavior AS segments
          FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
        ), UNNEST(SPLIT(segments, ',')) AS segment
      ),
      (
        --occasion_timing
        SELECT JSON_OBJECT('occasion_timing', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
        FROM (
          SELECT DISTINCT  occasion_timing AS segments
          FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
        ), UNNEST(SPLIT(segments, ',')) AS segment
      ),
      (
        --purchase_history
        SELECT JSON_OBJECT('purchase_history', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
        FROM (
          SELECT DISTINCT  purchase_history AS segments
          FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
        ), UNNEST(SPLIT(segments, ',')) AS segment
      ),
      (
          --spending_habits
          SELECT JSON_OBJECT('spending_habits', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  spending_habits AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --usage_frequency
          SELECT JSON_OBJECT('usage_frequency', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  usage_frequency AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --user_status
          SELECT JSON_OBJECT('user_status', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  user_status AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --age
          SELECT JSON_OBJECT('age', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  age AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --education
          SELECT JSON_OBJECT('education', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  education AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --ethnicity
          SELECT JSON_OBJECT('ethnicity', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  ethnicity AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --family_size
          SELECT JSON_OBJECT('family_size', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  family_size AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --gender
          SELECT JSON_OBJECT('gender', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  gender AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --generation
          SELECT JSON_OBJECT('generation', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  generation AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --income
          SELECT JSON_OBJECT('income', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  income AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
            WHERE income NOT LIKE 'Unknown%'
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --language
          SELECT JSON_OBJECT('language', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  language AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --marital_status
          SELECT JSON_OBJECT('marital_status', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  marital_status AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --occupation
          SELECT JSON_OBJECT('occupation', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  occupation AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --challenges
          SELECT JSON_OBJECT('challenges', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  challenges AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --goals
          SELECT JSON_OBJECT('goals', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  goals AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --pain_points
          SELECT JSON_OBJECT('pain_points', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  pain_points AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --priorities
          SELECT JSON_OBJECT('priorities', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  priorities AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --specific_needs
          SELECT JSON_OBJECT('specific_needs', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  specific_needs AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --interests
          SELECT JSON_OBJECT('interests', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  interests AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --lifestyle
          SELECT JSON_OBJECT('lifestyle', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  lifestyle AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --motivations
          SELECT JSON_OBJECT('motivations', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  motivations AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --personality
          SELECT JSON_OBJECT('personality', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  personality AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --customer_values
          SELECT JSON_OBJECT('customer_values', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  customer_values AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --social_media_platforms
          SELECT JSON_OBJECT('social_media_platforms', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  social_media_platforms AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        ),

        (
          --tech_savviness
          SELECT JSON_OBJECT('tech_savviness', ARRAY_AGG(DISTINCT TRIM(segment, " ")))
          FROM (
            SELECT DISTINCT  tech_savviness AS segments
            FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
          ), UNNEST(SPLIT(segments, ',')) AS segment
        )
    )
  )"""

  result = RunQuery(sql)
  return result

#### GetMenuItem(menu_id)

In [ ]:
def GetMenuItem(menu_id):
  sql = f"""SELECT *
  FROM `${project_id}.${bigquery_chocolate_ai_dataset}.menu`
  WHERE menu_id = {menu_id}
  """
  result = RunQuery(sql)
  return result

#### GetSegmentsFromCampaign()

In [ ]:
def GetSegmentsFromCampaign(campaign):

  # Extract campaign variables from the campaign object
  campaign_id = campaign['campaign_id']
  menu_id = campaign['menu_id']
  campaign_name = campaign['campaign_name']
  campaign_description = campaign['campaign_description']
  campaign_goal = campaign['campaign_goal']
  target_audience = campaign['target_audience']
  marketing_channels = campaign['marketing_channels']
  budget = campaign['budget']
  explanation = campaign['explanation']
  campaign_outcomes = campaign['campaign_outcomes']
  campaign_start_date = campaign['campaign_start_date']
  campaign_end_date = campaign['campaign_end_date']
  campaign_created_date = campaign['campaign_created_date']

  # Get menu item details
  menu_item = GetMenuItem(menu_id)
  menu_item = menu_item.iloc[0]
  menu_item_name = menu_item['menu_name']
  menu_item_description = menu_item['menu_description']
  menu_item_price = menu_item['menu_price']

  # Get pre-defined segments from the customer_marketing_profile table
  pre_defined_segments = GetPreDefinedSegments()
  pre_defined_segments = pre_defined_segments.iloc[0,0]

  # Define prompt
  prompt = f"""You are a Marketing Analyst, and you are reviewing Chocolate AI's latest
  marketing campaign called: {campaign_name}. Your task is to determine the ideal
  customer segments that should be targeted with the new campaign. You should choose
  at least two segments from the following set of pre-defined customer segments:

  {pre_defined_segments}

  You should also define one segment that doesn't exist in the pre-defined set of
  segments that describes the perfect target audience for the new campaign.

  Here are the details of the campaign:

  Campaign Name: {campaign_name}
  Campaign Description: {campaign_description}
  Menu Item being Promoted:
  Campaign Goal: {campaign_goal}
  Target Audience: {target_audience}
  Marketing Channels: {marketing_channels}
  Budget: {budget}
  Explanation: {explanation}
  Campaign Outcomes: {campaign_outcomes}
  Campaign Start Date: {campaign_start_date}
  Campaign End Date: {campaign_end_date}

  Now choose at least two segments from the set of pre-defined customer segments,
  and come up with one segment that doesn't exist in the pre-defined set of segments
  that describes the perfect target audience for the new campaign.

  Think step by step and explain your reasoning."""

  response_schema = {
    "type": "object",
    "required": [
      "pre_defined_segments", "novel_segment", "explanation"
    ],
    "properties": {
      "pre_defined_segments": {
        "type": "array",
        "description": "An array of customer segments chosen from a pre-defined list.",
        "items": {
          "type": "string",
          "description": "A segment that is chosen from a pre-defined list."
        }
      },
      "novel_segment": {
        "type": "string",
        "description": "A segment that is created to fit the ideal audience and does not exist in the pre-defined list."
      },
      "explanation": {
        "type": "string",
        "description": "An explanation for your reasoning on why you picked these segments."
      }
    }
  }

  result = GeminiLLM(prompt, response_schema=response_schema)
  return result

#### GetSegmentsFromVideo(campaign, uri)

In [ ]:
def GetSegmentsFromVideo(video_uri):

  # Get pre-defined segments from the customer_marketing_profile table
  pre_defined_segments = GetPreDefinedSegments()
  pre_defined_segments = pre_defined_segments.iloc[0,0]

  # Define prompt
  text_prompt = f"""You are a marketing analyst, and you are reviewing a new video
  ad created by your Content & Creatives team. Your task is to watch the new ad
  and determine the customer segments that would be most receptive to the new ad
  and also the most likely to make a purchase after seeing the ad.

  You should choose at least two segments from the following set of pre-defined
  customer segments:

  {pre_defined_segments}

  You should also define one segment that doesn't exist in the pre-defined set of
  segments that describes the perfect target audience for the new campaign.

  Now watch the supplied video, then choose at least two segments from the set of
  pre-defined customer segments, and come up with one segment that doesn't exist
  in the pre-defined set of segments that describes the perfect target audience
  for the new ad.

  Think step by step and explain your reasoning."""

  response_schema = {
    "type": "object",
    "required": [
      "pre_defined_segments", "novel_segment", "explanation"
    ],
    "properties": {
      "pre_defined_segments": {
        "type": "array",
        "description": "An array of customer segments chosen from a pre-defined list.",
        "items": {
          "type": "string",
          "description": "A segment that is chosen from a pre-defined list."
        }
      },
      "novel_segment": {
        "type": "string",
        "description": "A segment that is created to fit the ideal audience and does not exist in the pre-defined list."
      },
      "explanation": {
        "type": "string",
        "description": "An explanation for your reasoning on why you picked these segments."
      }
    }
  }

  response = GeminiLLM_Multimodal(text_prompt, video_uri, response_schema=response_schema)

  # Display the results
  #response_text = response['candidates'][0]['content']['parts'][0]['text']

  #result = GeminiLLM(prompt, response_schema=response_schema)
  return response

## <font color='#4285f4'>Get Segments from Campaign Description</font>

In [ ]:
# Get latest generated campaign
sql = """SELECT * FROM `${project_id}.${bigquery_chocolate_ai_dataset}.campaign`
         ORDER BY campaign_created_date
         LIMIT 1;"""

campaign = RunQuery(sql)
campaign = campaign.iloc[0]

recommended_segments = GetSegmentsFromCampaign(campaign)
recommended_segments = json.loads(recommended_segments)


In [ ]:
result_string = f"""
##Suggested Customer Segments

**Campaign Name:**
{campaign['campaign_name']}

**Campaign Description:**
{campaign['campaign_description']}

**Suggested customer segments (existing):**
{recommended_segments["pre_defined_segments"]}

**Ideal customer segment (new segment):**
{recommended_segments["novel_segment"]}

**Reasoning for segment choices:**
{recommended_segments['explanation']}
"""

DisplayMarkdown(result_string)


Let's assume that Gemini chose the following values:

- Existing customer segments to target:  `["purchase_history": "High Spender"', '"social_media_platforms": ["Instagram", "Facebook"]]`
- Ideal customer segment: "Luxury Foodie: Individuals with discerning palates who seek unique and high-quality culinary experiences. They are interested in the story behind their food, appreciate artisanal craftsmanship, and are willing to spend a premium for exceptional taste and ingredients."

We can search through our customer data to get a list of customers we should target for this campaign based on this criteria.

In [ ]:
sql = """SELECT COUNT(*) AS prospect_count
         FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
         WHERE purchase_history = 'High Spender'
         OR purchase_history = 'Frequent Buyer';"""

prospect_count = RunQuery(sql)
prospect_count = prospect_count.iloc[0]

print(f"There are {prospect_count['prospect_count']} customers in the recommended pre-defined segments.")

sql = """SELECT *
         FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
         WHERE purchase_history = 'High Spender'
         OR purchase_history = 'Frequent Buyer';"""

prospect_list = RunQuery(sql)
prospect_list

We can also use Gemini to extract the key words from the description of the new ideal customer segment, and we can use vector search on our customer_marketing_profile data to get a list of customers who match this criteria.

In [ ]:
prompt = f"""Extract up to 7 key words from the following customer segment description:

{recommended_segments["novel_segment"]}
"""

response_schema = {
    "type": "object",
    "required": [
      "key_words"
    ],
    "properties": {
      "key_words": {
        "type": "array",
        "description": "An array of key words extracted from the supplied text.",
        "items": {
          "type": "string",
          "description": "A key term extracted from the supplied text."
        }
      }
    }
  }

key_words = GeminiLLM(prompt, response_schema=response_schema)
key_words = json.loads(key_words)
key_words_string = " ".join(key_words['key_words'])
print(f"Key words: {key_words_string}")

In [ ]:
# Get the top 1000 customers that match the target customer segment
row_limit = 1000
sql = f"""WITH target_customers AS (
  SELECT
    distance,
    base.customer_id,
    base.customer_marketing_insights,
    base.customer_profile_data,
    base.customer_loyalty_data,
    base.customer_segmentation_data
    --base.customer_marketing_insights_embedding
  FROM VECTOR_SEARCH(
    -- base table or subquery
    TABLE `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile`,

    -- embedding column to search in base table - must be of type ARRAY<FLOAT64>
    'customer_marketing_insights_embedding',

    -- query table or subquery - this is where you generate the search embedding
    (
      SELECT ml_generate_embedding_result, content AS query
      FROM ML.GENERATE_EMBEDDING(
        MODEL `${project_id}.${bigquery_chocolate_ai_dataset}.textembedding_model`,
          (
            -- Search term
            SELECT "{key_words_string}" AS content
          ),
          STRUCT(
            TRUE AS flatten_json_output,
            'SEMANTIC_SIMILARITY' as task_type,
            768 AS output_dimensionality
          )
      )
    ),
    top_k => {row_limit}
  ) )
  SELECT distance,
    target_customers.customer_id,
    cus.customer_name,
    cus.customer_email,
    cus.customer_yob,
    target_customers.customer_marketing_insights,
    target_customers.customer_profile_data,
    target_customers.customer_loyalty_data,
    target_customers.customer_segmentation_data
  FROM target_customers
  JOIN `${project_id}.${bigquery_chocolate_ai_dataset}.customer` cus
  ON target_customers.customer_id = cus.customer_id
  ORDER BY distance;
  """

result = RunQuery(sql)
result

## <font color='#4285f4'>Get Segments from Video</font>

### Copy Video to Local Bucket

In [ ]:
# Copy the sample ad video to your local bucket
source_video_uri = "gs://data-analytics-golden-demo/chocolate-ai/v1/Campaign-Assets-Text-to-Video-01/story-01/full-video-with-audio-en-GB.mp4"
target_video_uri = f"gs://{bucket_name}/chocolate_ai/videos/chocolate-ai_story-HITL-01_full-video-with-audio-en-GB.mp4"

!gsutil -m cp  {source_video_uri} {target_video_uri}


### Watch the New Marketing Video

In [ ]:
# View the video
uri_index = target_video_uri.index(bucket_name)
video_uri = 'https://storage.cloud.google.com/' + target_video_uri[uri_index:]

HTML(f"""
<video width=600 height=337 controls autoplay>
      <source src="{video_uri}" type="video/mp4">
</video>
""")

### Get Recommended Segments for Ad from Gemini

In [ ]:
# Have Gemini watch the ad and suggest target segments
result = GetSegmentsFromVideo(target_video_uri)

json_result = json.loads(result['candidates'][0]['content']['parts'][0]['text'])
pre_defined_segments = json_result['pre_defined_segments']
novel_segment = json_result['novel_segment']
explanation = json_result['explanation']

result_string = f"""
##Suggested Customer Segments for Video

**Suggested customer segments (existing):**
{json_result["pre_defined_segments"]}

**Ideal customer segment (new segment):**
{json_result["novel_segment"]}

**Reasoning for segment choices:**
{json_result['explanation']}
"""

DisplayMarkdown(result_string)

Let's assume that Gemini chose the following values:

- Existing customer segments to target: ['Special Occasions', 'High Spender']
- Ideal customer segment: "Luxury Dessert Enthusiast"

We can search through our customer data to get a list of customers we should target for this campaign based on this criteria.

In [ ]:
sql = """SELECT COUNT(*) AS prospect_count
         FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
         WHERE purchase_history = 'High Spender'
         OR occasion_timing = 'Special Occasions';"""

prospect_count = RunQuery(sql)
prospect_count = prospect_count.iloc[0]

print(f"There are {prospect_count['prospect_count']} customers in the recommended pre-defined segments.")

sql = """SELECT *
         FROM `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile_segments`
         WHERE purchase_history = 'High Spender'
         OR occasion_timing = 'Special Occasions';"""

prospect_list = RunQuery(sql)
prospect_list

We can also use Gemini to generate key words related to the new ideal customer segment, and we can use vector search on our customer_marketing_profile data to get a list of customers who match this criteria.

In [ ]:
prompt = f"""Generate up to 7 key words that describe a customer who fits into the following customer segment:

{novel_segment}
"""

response_schema = {
    "type": "object",
    "required": [
      "key_words"
    ],
    "properties": {
      "key_words": {
        "type": "array",
        "description": "An array of key words extracted from the supplied text.",
        "items": {
          "type": "string",
          "description": "A key term extracted from the supplied text."
        }
      }
    }
  }

key_words = GeminiLLM(prompt, response_schema=response_schema)
key_words = json.loads(key_words)
key_words_string = " ".join(key_words['key_words'])
print(f"Key words: {key_words_string}")

In [ ]:
# Get the top 1000 customers that match the target customer segment
row_limit = 1000
sql = f"""WITH target_customers AS (
  SELECT
    distance,
    base.customer_id,
    base.customer_marketing_insights,
    base.customer_profile_data,
    base.customer_loyalty_data,
    base.customer_segmentation_data
    --base.customer_marketing_insights_embedding
  FROM VECTOR_SEARCH(
    -- base table or subquery
    TABLE `${project_id}.${bigquery_chocolate_ai_dataset}.customer_marketing_profile`,

    -- embedding column to search in base table - must be of type ARRAY<FLOAT64>
    'customer_marketing_insights_embedding',

    -- query table or subquery - this is where you generate the search embedding
    (
      SELECT ml_generate_embedding_result, content AS query
      FROM ML.GENERATE_EMBEDDING(
        MODEL `${project_id}.${bigquery_chocolate_ai_dataset}.textembedding_model`,
          (
            -- Search term
            SELECT "{key_words_string}" AS content
          ),
          STRUCT(
            TRUE AS flatten_json_output,
            'SEMANTIC_SIMILARITY' as task_type,
            768 AS output_dimensionality
          )
      )
    ),
    top_k => {row_limit}
  ) )
  SELECT distance,
    target_customers.customer_id,
    cus.customer_name,
    cus.customer_email,
    cus.customer_yob,
    target_customers.customer_marketing_insights,
    target_customers.customer_profile_data,
    target_customers.customer_loyalty_data,
    target_customers.customer_segmentation_data
  FROM target_customers
  JOIN `${project_id}.${bigquery_chocolate_ai_dataset}.customer` cus
  ON target_customers.customer_id = cus.customer_id
  ORDER BY distance ASC;
  """

result = RunQuery(sql)
result

## <font color='#4285f4'>Reference Links</font>


- [Generate multimodal content with the Gemini Enterprise API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference)
- [BigQuery JSON Functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/json_functions)
- [Controlled Generation with Gemini](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output)
- [Working with JSON Data in BigQuery](https://cloud.google.com/bigquery/docs/json-data)
- [Generate Embeddings in BigQuery with ML.GENERATE_EMBEDDING](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-embedding)
- [BigQuery Vector Indexes](https://cloud.google.com/bigquery/docs/vector-index)
- [BigQuery VECTOR_SEARCH() Function](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search)